# Section 2: Signals & Decision Engine

**Goal**: NSFW scalar + OCR keywords + rule engine + end-to-end decision helper.

**Scope**: Days 3-7 of MVP build. Combine hash matching with ML classifiers and policy rules.

**Links**:
- OpenNSFW2: https://github.com/bhky/opennsfw2
- PaddleOCR: https://github.com/PaddlePaddle/PaddleOCR
- TikTok Content Levels: https://support.tiktok.com/en/safety-hc/account-and-user-safety/content-levels-on-tiktok-posts

In [ ]:
%pip install --quiet torch torchvision opennsfw2 paddleocr paddlepaddle pillow numpy

import torch
import torchvision
import opennsfw2
import paddleocr
from PIL import Image
import numpy as np
import time
import json
import re
import os

print(f"PyTorch version: {torch.__version__}")
print(f"OpenNSFW2 version: {opennsfw2.__version__}")
print(f"PaddleOCR version: {paddleocr.__version__}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
_nsfw_model = None

def load_nsfw_model():
    global _nsfw_model
    if _nsfw_model is None:
        _nsfw_model = opennsfw2.make_open_nsfw_model()
        _nsfw_model.eval()
        if torch.cuda.is_available():
            _nsfw_model = _nsfw_model.cuda()
    return _nsfw_model

def score_nsfw(image_path):
    try:
        model = load_nsfw_model()
        image = Image.open(image_path).convert('RGB')
        inputs = opennsfw2.preprocess_image(image, opennsfw2.Preprocessing.YAHOO)
        
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            
        with torch.no_grad():
            outputs = model(inputs)
            nsfw_probability = torch.sigmoid(outputs[0]).cpu().item()
        return float(nsfw_probability)
    except Exception as e:
        print(f"NSFW scoring failed for {image_path}: {e}")
        return 0.0

In [ ]:
_ocr_model = None

def load_ocr_model():
    global _ocr_model
    if _ocr_model is None:
        _ocr_model = paddleocr.PaddleOCR(use_angle_cls=True, lang='en', show_log=False)
    return _ocr_model

def extract_text(image_path):
    try:
        ocr = load_ocr_model()
        result = ocr.ocr(image_path, cls=True)
        if result[0] is None:
            return ""
        
        text_parts = []
        for line in result[0]:
            if len(line) >= 2 and line[1][1] > 0.5:
                text_parts.append(line[1][0])
        
        return " ".join(text_parts).strip()
    except Exception as e:
        print(f"OCR failed for {image_path}: {e}")
        return ""

HATE_KEYWORDS = ["slur1", "slur2", "nazi", "terrorist", "hate"]
THREAT_KEYWORDS = ["kill", "shoot", "bomb", "murder", "attack"]
SEXTORTION_KEYWORDS = ["pay", "bitcoin", "leak", "expose", "money", "send"]

def flag_keywords(text):
    if not text:
        return {"hate": False, "threat": False, "sextortion": False}
    
    text_lower = text.lower()
    
    hate_flag = any(keyword in text_lower for keyword in HATE_KEYWORDS)
    threat_flag = any(keyword in text_lower for keyword in THREAT_KEYWORDS)
    sextortion_flag = any(keyword in text_lower for keyword in SEXTORTION_KEYWORDS)
    
    return {
        "hate": hate_flag,
        "threat": threat_flag,
        "sextortion": sextortion_flag
    }

In [ ]:
HASH_DB = {
    "threat_feed:1234567890ab": {"hash": "1234567890abcdef" + "0" * 48, "quality": 100, "labels": {"CSAM"}},
    "threat_feed:fedcba098765": {"hash": "fedcba0987654321" + "0" * 48, "quality": 100, "labels": {"TERROR"}},
    "threat_feed:abcdef123456": {"hash": "abcdef1234567890" + "0" * 48, "quality": 100, "labels": {"NCII"}}
}

def compute_pdq(image_path):
    try:
        import pdqhash
        with Image.open(image_path) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            hash_int, quality = pdqhash.compute(img)
            hash_hex = f"{hash_int:064x}"
            return hash_hex, quality
    except:
        import hashlib
        with open(image_path, 'rb') as f:
            content = f.read()
        hash_hex = hashlib.md5(content).hexdigest().ljust(64, '0')
        return hash_hex, 50

def hamming_distance_hex(hex1, hex2):
    return bin(int(hex1, 16) ^ int(hex2, 16)).count('1')

def match_hash(query_hash, max_distance=30, topk=50):
    matches = []
    for media_id, data in HASH_DB.items():
        distance = hamming_distance_hex(query_hash, data["hash"])
        if distance <= max_distance:
            matches.append((media_id, distance, data.get("labels", set())))
    
    matches.sort(key=lambda x: x[1])
    return matches[:topk]

In [ ]:
POLICY = {
    "hash": {
        "max_distance": 30
    },
    "nsfw": {
        "threshold_adult_blur": 0.95,
        "threshold_teen_block": 0.85,
        "threshold_child_block": 0.70
    },
    "ocr": {
        "enable": True,
        "require_review_on_hate": True,
        "require_review_on_threat": True,
        "require_review_on_sextortion": True,
        "confidence_threshold": 0.7
    },
    "age": {
        "teen_mode": False,
        "strict_mode": False
    },
    "surfaces": {
        "feed": {"nsfw_multiplier": 1.0, "strictness": "normal"},
        "dm": {"nsfw_multiplier": 0.8, "strictness": "relaxed"},
        "avatar": {"nsfw_multiplier": 1.2, "strictness": "strict"},
        "story": {"nsfw_multiplier": 0.9, "strictness": "normal"}
    }
}

def decide(image_path, matches, nsfw_p, ocr_text, flags, teen_mode=False, surface="feed"):
    reasons = []
    action = "ALLOW"
    confidence = 0.0
    
    surface_config = POLICY["surfaces"].get(surface, POLICY["surfaces"]["feed"])
    nsfw_adjusted = nsfw_p * surface_config["nsfw_multiplier"]
    
    harmful_labels = {"CSAM", "NCII", "TERROR"}
    for media_id, distance, labels in matches:
        if distance <= POLICY["hash"]["max_distance"] and labels & harmful_labels:
            action = "BLOCK"
            confidence = 1.0
            reasons.append(f"Hash match: {list(labels)} (distance: {distance})")
            break
    
    if action == "ALLOW":
        if teen_mode:
            if nsfw_adjusted >= POLICY["nsfw"]["threshold_teen_block"]:
                action = "BLOCK"
                confidence = min(1.0, nsfw_adjusted)
                reasons.append(f"Teen NSFW block (score: {nsfw_adjusted:.3f})")
            elif nsfw_adjusted >= POLICY["nsfw"]["threshold_child_block"]:
                action = "BLUR"
                confidence = min(1.0, nsfw_adjusted)
                reasons.append(f"Teen NSFW blur (score: {nsfw_adjusted:.3f})")
        else:
            if nsfw_adjusted >= POLICY["nsfw"]["threshold_adult_blur"]:
                action = "BLUR"
                confidence = min(1.0, nsfw_adjusted)
                reasons.append(f"Adult NSFW blur (score: {nsfw_adjusted:.3f})")
    
    if action == "ALLOW" and POLICY["ocr"]["enable"]:
        review_flags = []
        if flags.get("hate") and POLICY["ocr"]["require_review_on_hate"]:
            review_flags.append("hate")
        if flags.get("threat") and POLICY["ocr"]["require_review_on_threat"]:
            review_flags.append("threat")
        if flags.get("sextortion") and POLICY["ocr"]["require_review_on_sextortion"]:
            review_flags.append("sextortion")
        
        if review_flags:
            action = "REVIEW"
            confidence = 0.8
            reasons.append(f"OCR flags: {review_flags}")
    
    if surface_config["strictness"] == "strict" and action == "ALLOW" and nsfw_p > 0.5:
        action = "REVIEW"
        confidence = nsfw_p
        reasons.append(f"Strict surface review (score: {nsfw_p:.3f})")
    
    return {
        "action": action,
        "reasons": reasons,
        "nsfw_p": nsfw_p,
        "nsfw_adjusted": nsfw_adjusted,
        "confidence": confidence,
        "surface": surface,
        "matches": [(mid, dist) for mid, dist, _ in matches],
        "ocr_excerpt": ocr_text[:100] + "..." if len(ocr_text) > 100 else ocr_text
    }

def update_policy(updates):
    for key, value in updates.items():
        if key in POLICY:
            if isinstance(POLICY[key], dict) and isinstance(value, dict):
                POLICY[key].update(value)
            else:
                POLICY[key] = value
    return POLICY.copy()

def get_policy_summary():
    return {
        "hash_distance": POLICY["hash"]["max_distance"],
        "nsfw_thresholds": POLICY["nsfw"],
        "ocr_enabled": POLICY["ocr"]["enable"],
        "surfaces": list(POLICY["surfaces"].keys()),
        "current_mode": "teen" if POLICY["age"]["teen_mode"] else "adult"
    }

In [ ]:
def run_decision(image_path, max_distance=None, teen_mode=None, surface="feed"):
    if max_distance is None:
        max_distance = POLICY["hash"]["max_distance"]
    if teen_mode is None:
        teen_mode = POLICY["age"]["teen_mode"]
    
    start_time = time.time()
    
    hash_hex, quality = compute_pdq(image_path)
    matches = match_hash(hash_hex, max_distance)
    
    nsfw_p = score_nsfw(image_path)
    
    ocr_text = extract_text(image_path)
    flags = flag_keywords(ocr_text)
    
    decision = decide(image_path, matches, nsfw_p, ocr_text, flags, teen_mode, surface)
    
    end_time = time.time()
    
    result = {
        "image_path": image_path,
        "hash_hex": hash_hex,
        "hash_quality": quality,
        "processing_time_ms": int((end_time - start_time) * 1000),
        "teen_mode": teen_mode,
        "ocr_flags": flags,
        **decision
    }
    
    return result

def test_policy_scenarios():
    print("\\nTesting policy scenarios across surfaces and age groups...")
    
    surfaces = ["feed", "dm", "avatar", "story"]
    age_modes = [False, True]  # Adult, Teen
    
    results = {}
    
    for surface in surfaces:
        for teen_mode in age_modes:
            age_label = "teen" if teen_mode else "adult"
            key = f"{surface}_{age_label}"
            
            result = run_decision('test_images/nsfw.jpg', teen_mode=teen_mode, surface=surface)
            results[key] = {
                "action": result["action"],
                "nsfw_p": result["nsfw_p"],
                "nsfw_adjusted": result.get("nsfw_adjusted", result["nsfw_p"]),
                "confidence": result.get("confidence", 0.0)
            }
            
    for key, result in results.items():
        print(f"  {key}: {result['action']} (NSFW: {result['nsfw_p']:.3f}→{result['nsfw_adjusted']:.3f}, conf: {result['confidence']:.3f})")
    
    return results

In [ ]:
def batch_decide(image_paths):
    results = []
    for path in image_paths:
        try:
            result = run_decision(path)
            results.append(result)
        except Exception as e:
            results.append({
                "image_path": path,
                "action": "ERROR",
                "reasons": [f"Processing failed: {str(e)}"],
                "nsfw_p": 0.0,
                "processing_time_ms": 0,
                "matches": []
            })
    
    print(f"{'Image':<30} {'Action':<10} {'NSFW':<8} {'Min Dist':<8} {'Time(ms)':<10}")
    print("-" * 70)
    
    for result in results:
        image_name = os.path.basename(result["image_path"])[:28]
        action = result["action"]
        nsfw_p = f"{result['nsfw_p']:.3f}"
        
        matches = result.get("matches", [])
        min_dist = str(min([d for _, d in matches], default="N/A"))
        
        time_ms = str(result.get("processing_time_ms", 0))
        
        print(f"{image_name:<30} {action:<10} {nsfw_p:<8} {min_dist:<8} {time_ms:<10}")
    
    return results

def create_test_image(path, text=None, color=(255, 255, 255), size=(200, 100)):
    from PIL import ImageDraw, ImageFont
    img = Image.new('RGB', size, color)
    if text:
        draw = ImageDraw.Draw(img)
        try:
            font = ImageFont.truetype("/System/Library/Fonts/Arial.ttf", 20)
        except:
            font = ImageFont.load_default()
        draw.text((10, 40), text, fill=(0, 0, 0), font=font)
    img.save(path)

os.makedirs('test_images', exist_ok=True)
create_test_image('test_images/clean.jpg', 'Hello World')
create_test_image('test_images/threat.jpg', 'I will kill you')
create_test_image('test_images/sextortion.jpg', 'Pay me bitcoin or I leak')

test_paths = ['test_images/clean.jpg', 'test_images/threat.jpg', 'test_images/sextortion.jpg']
print("Testing batch decision on sample images:")
batch_results = batch_decide(test_paths)

os.makedirs('test_images', exist_ok=True)
create_test_image('test_images/clean.jpg', 'Hello World')
create_test_image('test_images/threat.jpg', 'I will kill you')
create_test_image('test_images/sextortion.jpg', 'Pay me bitcoin or I leak')
create_test_image('test_images/nsfw.jpg', 'Adult Content', (255, 100, 100))

test_paths = ['test_images/clean.jpg', 'test_images/threat.jpg', 'test_images/sextortion.jpg', 'test_images/nsfw.jpg']
print("Testing batch decision on sample images:")
batch_results = batch_decide(test_paths)

print("\\nPolicy Summary:")
policy_summary = get_policy_summary()
for key, value in policy_summary.items():
    print(f"  {key}: {value}")

policy_test_results = test_policy_scenarios()

print("\\nTesting policy updates...")
original_policy = POLICY.copy()
update_policy({"nsfw": {"threshold_adult_blur": 0.8}})
print(f"Updated adult blur threshold to: {POLICY['nsfw']['threshold_adult_blur']}")

updated_result = run_decision('test_images/nsfw.jpg')
print(f"New result with updated policy: {updated_result['action']} (was BLUR, now with lower threshold)")

POLICY.update(original_policy)
print("Policy restored to original values")